In [1]:
from Synthesizer import synthesize

In [114]:
import pandas as pd

DF = pd.DataFrame


class WrongPathError(Exception):
    pass


class TableManager:
    def __init__(self, df: DF):
        self.df = df
    
    def _repr_html_(self):
        return self.df._repr_html_()
    
    @property
    def _columns(self) -> list:
        return self.df.columns
    
    def _len(self, x_axis: bool) -> int:
        return self.df.shape[1 if x_axis else 0]

    def _check_relative_index(self, index: int, x_axis: bool) -> None:

        if abs(index) > self._len(x_axis) / 2:
            raise WrongPathError("Index is absolute. Should use relative index.")
    
    def _check_inbound_index(self, index: int, x_axis: bool) -> None:
        total_len = self._len(x_axis)

        if index >= total_len or index < -total_len:
            raise WrongPathError("Index is out of bound.")
    
    def _copy_modify(self, function) -> DF:
        return TableManager(function(self.df))
    
    def _copy_getitem(self, key) -> DF:
        return self._copy_modify(lambda table: table.__getitem__(key))
    
    def _copy_loc(self, key1, key2 = None) -> DF:
        if key2 is None:
            return self._copy_modify(lambda table: table.loc[key1])
        else:
            return self._copy_modify(lambda table: table.loc[key1, key2])
    
    def _to_absolute_index(self, index: int, x_axis: bool) -> int:
        return index if index >= 0 else self._len(x_axis) + index
    
    def _check_valid_start_end(self, start: int, end: int, x_axis: bool) -> None:
        start = self._to_absolute_index(start, x_axis)
        end = self._to_absolute_index(end, x_axis)

        if start >= end:
            raise WrongPathError("Start index is greater than or equal to end index.")
        
        self._check_inbound_index(start, x_axis)
        self._check_inbound_index(end-1, x_axis)

    def reorder(self, start: int, end: int, move: int, x_axis: bool) -> DF:
        self._check_relative_index(start, x_axis)
        self._check_relative_index(end, x_axis)
        
        self._check_valid_start_end(start, end, x_axis)
        
        if move == 0:
            raise WrongPathError("Move is 0. Should be an effective move.")
        elif move > 0:
            self._check_inbound_index(end-1+move, x_axis)
        else:
            self._check_inbound_index(start+move, x_axis)

        return self._x_reorder(start, end, move) if x_axis else self._y_reorder(start, end, move)
    
    def _x_reorder(self, start: int, end: int, move: int) -> DF:
        cols = list(self._columns)
        cols_to_move = cols[start:end]
        del cols[start:end]
        for i, col in enumerate(cols_to_move):
            cols.insert(start + move + i, col)
        return self._copy_getitem(cols)
    
    def _y_reorder(self, start: int, end: int, move: int) -> DF:
        idx = list(self.df.index)
        idx_to_move = idx[start:end]
        del idx[start:end]
        for i, index in enumerate(idx_to_move):
            idx.insert(start + move + i, index)
        return self._copy_loc(idx)


In [115]:
data = {'A': [1, 2, 3, 4], 'B': [5, 6, 7, 8], 'C': [9, 10, 11, 12], 'D': [13, 14, 15, 16]}
df = pd.DataFrame(data)
df

,A,B,C,D
0,1,5,9,13
1,2,6,10,14
2,3,7,11,15
3,4,8,12,16


In [119]:
teste = TableManager(df)
teste

,A,B,C,D
0,1,5,9,13
1,2,6,10,14
2,3,7,11,15
3,4,8,12,16


In [118]:
teste.reorder(0, 1, -2, False).reorder(1, 2, -1, False)

,A,B,C,D
0,1,5,9,13
1,2,6,10,14
2,3,7,11,15
3,4,8,12,16


In [2]:
programs = synthesize(
    input_output_simple_list=[
        ([1, 5, 2], [1, 5]),
        ([3, 9], [3])
    ],
    variable_names=['x'],
    search_space_file='utils/grammars/ListGrammar.txt',
    max_height=2,
    statistics=True
)

The synthesizer searched 52 programs up to height #1.


In [3]:
programs

['x[0:-1:None]']